# Script 3 to clean the C-CLAMP metadata file
This script further enriches the authors' countries of birth and death.

This script has to be available within the same folder as ...
1) `author_filtered_clean.txt`
2) `the folder scraped_json`

## Step 1: Extract the countries from the scraped JSON files

In [3]:
# Import the necessary packages
import json
import pandas as pd
import numpy as np
import os
from glob import glob

In [4]:
# Process the JSON files and extract all identifiers, countries of birth, and countries of death from them, if available
# Create a list to store all metadata
metadata = []

# Store all filenames
filenames = glob('scraped_json/*.json')

# Loop over the filenames, read the JSON files and extract the desired fields
# If a field cannot be found, the value is set to None
# All data is appended to the metadata list
for filename in filenames:
    with open(filename, 'r', encoding='utf-8') as file:
        try:
            data_json = json.load(file)
            try:
                birthCountry = data_json['birthPlace']['addressCountry']
            except KeyError:
                birthCountry = None

            try:
                deathCountry = data_json['deathPlace']['addressCountry']
            except KeyError:
                deathCountry = None

            try:
                identifier = data_json['identifier']
            except KeyError:
                identifier = None

            metadata.append({
                'identifier': identifier,
                'birthCountry_json': birthCountry,
                'deathCountry_json': deathCountry
            })

        except json.JSONDecodeError:
            print(f"Decoding failed: {filename}")

# Convert the metadata list to a DataFrame
metadata_countries_df = pd.DataFrame(metadata)
metadata_countries_df

,identifier,birthCountry_json,deathCountry_json
0,aafj001,Nederland,Nederland
1,aage001,Nederland,None
2,aaks001,None,None
3,aalb006,None,None
4,aald006,Nederland,Nederland
...,...,...,...
8725,_nie010,None,None
8726,_ont003,None,None
8727,_rid002,None,None
8728,_vad003,None,None


In [5]:
## Add missing data
# Manually add the data for 23 authors for which the scraping failed
metadata_countries_manual_df = pd.DataFrame({'identifier': ['enge016', 'swae001', 'pete023', 'will025', 'kamp057', 'verv028', 'lepe001', 'bem_001',
                                                            'lach007', 'draa001', 'roos006', 'weel003', 'domm004', 'baum009', 'bezo001', 'hard001',
                                                            'bred001', 'copp010', 'stro018', 'rode001', 'bran218', 'merc005', 'faas001'],
                                             'birthCountry_json': ['Nederland', 'Frankrijk', None, 'België (Wallonië)', 'Nederland', None, None,
                                                                   'Oekraïne', 'Nederland', 'Nederland', 'België (Wallonië)', 'Nederland', None,
                                                                   'Rusland', 'Nederland', 'België (Wallonië)', 'Nederland', None, None,
                                                                   'België (Wallonië)', None, None, 'Nederland'],
                                             'deathCountry_json': ['Nederland', 'Frankrijk', None, 'Nederland', 'Nederland', None, None, None,
                                                                   'Nederland', None, None, 'Nederland', None, None, 'Nederland', 'België (Wallonië)',
                                                                   'Nederland', None, None, 'België (Wallonië)', None, None, 'Nederland']})

# Concatenate both DataFrames
metadata_countries_full_df = pd.concat([metadata_countries_df, metadata_countries_manual_df])
metadata_countries_full_df

,identifier,birthCountry_json,deathCountry_json
0,aafj001,Nederland,Nederland
1,aage001,Nederland,None
2,aaks001,None,None
3,aalb006,None,None
4,aald006,Nederland,Nederland
...,...,...,...
18,stro018,None,None
19,rode001,België (Wallonië),België (Wallonië)
20,bran218,None,None
21,merc005,None,None


In [6]:
# Save the completed DataFrame
metadata_countries_full_df.to_csv("countries_json.txt", index=False, sep='\t', encoding='utf-8')

## Step 2: Add to the other metadata

In [8]:
# Read the previously cleaned metadata file
metadata_df = pd.read_csv("author_filtered_clean.txt", sep="\t", encoding="utf-8")

In [9]:
# Merge metadata_countries_full_df with metadata_df
metadata_full_df = pd.merge(metadata_df, metadata_countries_full_df, on='identifier', how='left')

print(metadata_full_df)
metadata_full_df.dtypes

                  Author                                               Link  \
0           Jan Engelman  https://www.dbnl.org/auteurs/auteur.php?id=eng...   
1            Willem Maas  https://www.dbnl.org/auteurs/auteur.php?id=maa...   
2           Joep Nicolas  https://www.dbnl.org/auteurs/auteur.php?id=nic...   
3          Albert Helman  https://www.dbnl.org/auteurs/auteur.php?id=hel...   
4     Willem Nieuwenhuis  https://www.dbnl.org/auteurs/auteur.php?id=nie...   
...                  ...                                                ...   
8748        A. De Geyter  https://www.dbnl.org/auteurs/auteur.php?id=gey...   
8749            J. Hoing  https://www.dbnl.org/auteurs/auteur.php?id=hoi...   
8750       Flor Kielbaey  https://www.dbnl.org/auteurs/auteur.php?id=kie...   
8751  Hendrik Imberechts  https://www.dbnl.org/auteurs/auteur.php?id=imb...   
8752            H. Aerts  https://www.dbnl.org/auteurs/auteur.php?id=aer...   

     identifier                              givenN

Author               object
Link                 object
identifier           object
givenName            object
familyName           object
alternateNames       object
birthDate            object
birthPlace           object
birthCountry         object
deathDate            object
deathPlace           object
deathCountry         object
gender               object
occupation           object
period               object
language             object
category             object
organisation         object
birthCountry_json    object
deathCountry_json    object
dtype: object

In [10]:
## Compare birthCountry with birthCountry_json, and deathCountry with deathCountry_json
# Define a function to compare two columns
def compare_two(row, col_A, col_B):
    value_A = row[col_A]
    value_B = row[col_B]

    if pd.isna(value_A):
        return value_B
    else:
        return value_A

def apply_two_comparison(df, col_A, col_B, new_col):
    df[new_col] = df.apply(compare_two, axis=1, col_A=col_A, col_B=col_B)
    return df

# Apply the comparisons
metadata_full_df = apply_two_comparison(metadata_full_df, 'birthCountry', 'birthCountry_json', 'birthCountry_clean')
metadata_full_df = apply_two_comparison(metadata_full_df, 'deathCountry', 'deathCountry_json', 'deathCountry_clean')

print(metadata_full_df.columns)
metadata_full_df

Index(['Author', 'Link', 'identifier', 'givenName', 'familyName',
       'alternateNames', 'birthDate', 'birthPlace', 'birthCountry',
       'deathDate', 'deathPlace', 'deathCountry', 'gender', 'occupation',
       'period', 'language', 'category', 'organisation', 'birthCountry_json',
       'deathCountry_json', 'birthCountry_clean', 'deathCountry_clean'],
      dtype='object')


,Author,Link,identifier,givenName,familyName,alternateNames,birthDate,birthPlace,birthCountry,deathDate,...,gender,occupation,period,language,category,organisation,birthCountry_json,deathCountry_json,birthCountry_clean,deathCountry_clean
0,Jan Engelman,https://www.dbnl.org/auteurs/auteur.php?id=eng...,enge016,Johannes Aloysius Antonius,Engelman,Jan Engelman,1900-06-07,Utrecht,NaN,1972-03-20,...,male,redacteur/journalist/vertaler,Twintigste Eeuw,nl,secundair/nonfictie,NaN,Nederland,Nederland,Nederland,Nederland
1,Willem Maas,https://www.dbnl.org/auteurs/auteur.php?id=maa...,maas024,Willem Arnoldus,Maas,Willem Maas,1897-04-28,Utrecht,NaN,1950-05-06,...,male,illustrator/tekenaar/beeldend kunstenaar,Twintigste Eeuw,NaN,NaN,NaN,Nederland,Nederland,Nederland,Nederland
2,Joep Nicolas,https://www.dbnl.org/auteurs/auteur.php?id=nic...,nico008,Josephus Antonius Hubertus Franciscus,Nicolas,Joep Nicolas,1897-10-06,Roermond,NaN,1972-07-25,...,male,schilder/ambachtsman,Twintigste Eeuw,NaN,nonfictie,NaN,Nederland,Nederland,Nederland,Nederland
3,Albert Helman,https://www.dbnl.org/auteurs/auteur.php?id=hel...,helm003,Albert,Helman,Albert Helman/Beckmesser/Brandaris/Floris Kapt...,1903-11-07,Paramaribo,Suriname,1996-07-10,...,male,musicus/journalist/politicus/redacteur/staatsm...,Twintigste Eeuw,nl,jeugdliteratuur/nonfictie/suriname/buitenland,NaN,Suriname,Nederland,Suriname,Nederland
4,Willem Nieuwenhuis,https://www.dbnl.org/auteurs/auteur.php?id=nie...,nieu047,Willem,Nieuwenhuis,Willem Nieuwenhuis,1886-xx-xx,NaN,NaN,1935-xx-xx,...,male,NaN,Twintigste Eeuw,nl,NaN,NaN,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8748,A. De Geyter,https://www.dbnl.org/auteurs/auteur.php?id=gey...,geyt003,A.,De Geyter,A. De Geyter,19xx-xx-xx,NaN,NaN,NaN,...,male,NaN,Twintigste Eeuw,NaN,secundair,NaN,None,None,None,None
8749,J. Hoing,https://www.dbnl.org/auteurs/auteur.php?id=hoi...,hoin002,J.,Hoing,J. Hoing,19xx-xx-xx,NaN,NaN,NaN,...,male,NaN,Twintigste Eeuw,NaN,secundair/nonfictie,NaN,None,None,None,None
8750,Flor Kielbaey,https://www.dbnl.org/auteurs/auteur.php?id=kie...,kiel012,Flor,Kielbaey,Flor Kielbaey,19xx-xx-xx,NaN,NaN,NaN,...,male,vertaler,Twintigste Eeuw,NaN,secundair,NaN,None,None,None,None
8751,Hendrik Imberechts,https://www.dbnl.org/auteurs/auteur.php?id=imb...,imbe003,Hendrik,Imberechts,Hendrik Imberechts,1922-04-13,Muizen,NaN,2012-02-03,...,male,rk-geestelijke/kloosterling,Twintigste Eeuw,nl,secundair/nonfictie,NaN,België (Wallonië),België (Wallonië),België (Wallonië),België (Wallonië)


In [11]:
## Filter the DataFrame and rename the columns
# Filter
filtered_metadata_df = metadata_full_df[['Author', 'Link', 'identifier', 'givenName', 'familyName', 'alternateNames',
                                    'birthDate', 'birthPlace', 'birthCountry_clean', 'deathDate', 'deathPlace', 'deathCountry_clean',
                                    'gender', 'occupation', 'period', 'language', 'category', 'organisation']]

# Rename columns
filtered_metadata_df = filtered_metadata_df.rename(columns = {'birthCountry_clean': 'birthCountry', 'deathCountry_clean': 'deathCountry'})

# Fill empty cells with NaN
filtered_metadata_df = filtered_metadata_df.replace('', np.nan)

filtered_metadata_df.head(10)

,Author,Link,identifier,givenName,familyName,alternateNames,birthDate,birthPlace,birthCountry,deathDate,deathPlace,deathCountry,gender,occupation,period,language,category,organisation
0,Jan Engelman,https://www.dbnl.org/auteurs/auteur.php?id=eng...,enge016,Johannes Aloysius Antonius,Engelman,Jan Engelman,1900-06-07,Utrecht,Nederland,1972-03-20,Amsterdam,Nederland,male,redacteur/journalist/vertaler,Twintigste Eeuw,nl,secundair/nonfictie,NaN
1,Willem Maas,https://www.dbnl.org/auteurs/auteur.php?id=maa...,maas024,Willem Arnoldus,Maas,Willem Maas,1897-04-28,Utrecht,Nederland,1950-05-06,Utrecht,Nederland,male,illustrator/tekenaar/beeldend kunstenaar,Twintigste Eeuw,NaN,NaN,NaN
2,Joep Nicolas,https://www.dbnl.org/auteurs/auteur.php?id=nic...,nico008,Josephus Antonius Hubertus Franciscus,Nicolas,Joep Nicolas,1897-10-06,Roermond,Nederland,1972-07-25,Steyl,Nederland,male,schilder/ambachtsman,Twintigste Eeuw,NaN,nonfictie,NaN
3,Albert Helman,https://www.dbnl.org/auteurs/auteur.php?id=hel...,helm003,Albert,Helman,Albert Helman/Beckmesser/Brandaris/Floris Kapt...,1903-11-07,Paramaribo,Suriname,1996-07-10,Amsterdam,Nederland,male,musicus/journalist/politicus/redacteur/staatsm...,Twintigste Eeuw,nl,jeugdliteratuur/nonfictie/suriname/buitenland,NaN
4,Willem Nieuwenhuis,https://www.dbnl.org/auteurs/auteur.php?id=nie...,nieu047,Willem,Nieuwenhuis,Willem Nieuwenhuis,1886-xx-xx,NaN,None,1935-xx-xx,NaN,None,male,NaN,Twintigste Eeuw,nl,NaN,NaN
5,Karel van den Oever,https://www.dbnl.org/auteurs/auteur.php?id=oev...,oeve003,Karel,van den Oever,Karel van den Oever,1879-11-19,Antwerpen,België (Wallonië),1926-10-06,Antwerpen,België (Wallonië),male,arbeider/werkman,Twintigste Eeuw,nl,secundair/jeugdliteratuur,NaN
6,Wies Moens,https://www.dbnl.org/auteurs/auteur.php?id=moe...,moen007,Aloys Caesar Antoon,Moens,Wies Moens,1898-01-28,Sint-Gillis-Bij-Dendermonde,België (Wallonië),1982-02-05,Geleen,Nederland,male,NaN,Twintigste Eeuw,nl,secundair,NaN
7,Eduard A. Serrarens,https://www.dbnl.org/auteurs/auteur.php?id=ser...,serr010,Eduard A.,Serrarens,Eduard A. Serrarens,1895-12-27,Maastricht,Nederland,1955-11-25,Roermond,Nederland,male,leraar/docent/hoogleraar,Twintigste Eeuw,nl,secundair,NaN
8,Henk Kuitenbrouwer,https://www.dbnl.org/auteurs/auteur.php?id=kui...,kuit001,Carel Hendrik Gerhardus Maria,Kuitenbrouwer,Henk Kuitenbrouwer,1903-11-20,Utrecht,Nederland,1974-08-03,Utrecht,Nederland,male,onderwijzer (basisschool)/redacteur/journalist,Twintigste Eeuw,NaN,jeugdliteratuur,NaN
9,Jan Bruna,https://www.dbnl.org/auteurs/auteur.php?id=bru...,brun020,Jan,Bruna,Jan Bruna,1876-08-17,NaN,None,1931-03-11,NaN,None,male,redacteur/journalist,Twintigste Eeuw,nl,secundair,NaN


## Step 3: Correct mistakes in the country values
All places in Belgium have been falsely attributed the value 'België (Wallonië)'. We manually correct this and make a distinction between Flanders (Vlaanderen), Wallonia (Wallonië), and Brussels (Brussel).

In [13]:
# Replace the value 'België (Wallonië)' by 'België (Vlaanderen)'
filtered_metadata_df = filtered_metadata_df.replace({'birthCountry': {'België (Wallonië)': 'België (Vlaanderen)'},
                                                    'deathCountry': {'België (Wallonië)': 'België (Vlaanderen)'}})

In [14]:
Wallonia = ['Ans', 'Aywaille', 'Braine-le-Comte', 'Châtelineau', 'Enghien', 'Erneuville', 'Flobecq', 'La Louvière', 'Lijsem', 'Luik', 'Miavoye-Anthée',
            'Namen', 'Ophain-Bois-Seigneur-Isaac', 'Rixensart', 'Rosières', 'Verviers']
Brussels = ['Anderlecht', 'Brussel', 'Elsene', 'Etterbeek', 'Jette', 'Koekelberg', 'Laken', 'Oudergem', 'Schaarbeek', 'Sint-Gillis',
            'Sint-Jans-Molenbeek', 'Sint-Lambrechts-Woluwe', 'Sint-Pieters-Jette', 'St-Joost-Ten-Noode', 'Ukkel', 'Vorst']

filtered_metadata_df.loc[filtered_metadata_df['birthPlace'].isin(Wallonia), 'birthCountry'] = 'België (Wallonië)'
filtered_metadata_df.loc[filtered_metadata_df['birthPlace'].isin(Brussels), 'birthCountry'] = 'België (Brussel)'

filtered_metadata_df.loc[filtered_metadata_df['deathPlace'].isin(Wallonia), 'deathCountry'] = 'België (Wallonië)'
filtered_metadata_df.loc[filtered_metadata_df['deathPlace'].isin(Brussels), 'deathCountry'] = 'België (Brussel)'

filtered_metadata_df

,Author,Link,identifier,givenName,familyName,alternateNames,birthDate,birthPlace,birthCountry,deathDate,deathPlace,deathCountry,gender,occupation,period,language,category,organisation
0,Jan Engelman,https://www.dbnl.org/auteurs/auteur.php?id=eng...,enge016,Johannes Aloysius Antonius,Engelman,Jan Engelman,1900-06-07,Utrecht,Nederland,1972-03-20,Amsterdam,Nederland,male,redacteur/journalist/vertaler,Twintigste Eeuw,nl,secundair/nonfictie,NaN
1,Willem Maas,https://www.dbnl.org/auteurs/auteur.php?id=maa...,maas024,Willem Arnoldus,Maas,Willem Maas,1897-04-28,Utrecht,Nederland,1950-05-06,Utrecht,Nederland,male,illustrator/tekenaar/beeldend kunstenaar,Twintigste Eeuw,NaN,NaN,NaN
2,Joep Nicolas,https://www.dbnl.org/auteurs/auteur.php?id=nic...,nico008,Josephus Antonius Hubertus Franciscus,Nicolas,Joep Nicolas,1897-10-06,Roermond,Nederland,1972-07-25,Steyl,Nederland,male,schilder/ambachtsman,Twintigste Eeuw,NaN,nonfictie,NaN
3,Albert Helman,https://www.dbnl.org/auteurs/auteur.php?id=hel...,helm003,Albert,Helman,Albert Helman/Beckmesser/Brandaris/Floris Kapt...,1903-11-07,Paramaribo,Suriname,1996-07-10,Amsterdam,Nederland,male,musicus/journalist/politicus/redacteur/staatsm...,Twintigste Eeuw,nl,jeugdliteratuur/nonfictie/suriname/buitenland,NaN
4,Willem Nieuwenhuis,https://www.dbnl.org/auteurs/auteur.php?id=nie...,nieu047,Willem,Nieuwenhuis,Willem Nieuwenhuis,1886-xx-xx,NaN,None,1935-xx-xx,NaN,None,male,NaN,Twintigste Eeuw,nl,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8748,A. De Geyter,https://www.dbnl.org/auteurs/auteur.php?id=gey...,geyt003,A.,De Geyter,A. De Geyter,19xx-xx-xx,NaN,None,NaN,NaN,None,male,NaN,Twintigste Eeuw,NaN,secundair,NaN
8749,J. Hoing,https://www.dbnl.org/auteurs/auteur.php?id=hoi...,hoin002,J.,Hoing,J. Hoing,19xx-xx-xx,NaN,None,NaN,NaN,None,male,NaN,Twintigste Eeuw,NaN,secundair/nonfictie,NaN
8750,Flor Kielbaey,https://www.dbnl.org/auteurs/auteur.php?id=kie...,kiel012,Flor,Kielbaey,Flor Kielbaey,19xx-xx-xx,NaN,None,NaN,NaN,None,male,vertaler,Twintigste Eeuw,NaN,secundair,NaN
8751,Hendrik Imberechts,https://www.dbnl.org/auteurs/auteur.php?id=imb...,imbe003,Hendrik,Imberechts,Hendrik Imberechts,1922-04-13,Muizen,België (Vlaanderen),2012-02-03,Leuven,België (Vlaanderen),male,rk-geestelijke/kloosterling,Twintigste Eeuw,nl,secundair/nonfictie,NaN


In [15]:
# Save the merged metadata file
filtered_metadata_df.to_csv("author_clean_final.txt", index=False, sep='\t', encoding='utf-8')